# Ingest group data
https://api.wonde.com/v1.0/schools/{school_id}/groups

In [14]:
%run _Config

StatementMeta(, e8215b12-1c89-4e48-9a67-2ae51d02e83a, -1, Finished, Available)

In [15]:
%run _WondeStructs

StatementMeta(, e8215b12-1c89-4e48-9a67-2ae51d02e83a, -1, Finished, Available)

In [16]:
%run _WondeAPI

StatementMeta(, e8215b12-1c89-4e48-9a67-2ae51d02e83a, -1, Finished, Available)

In [4]:
# Get basic json return
# url = "https://api.wonde.com/v1.0/schools/A1128526306/groups?include=students"
# session = requests.Session()
# session.get(url, headers=wondeHeaders).json()

StatementMeta(, cccf3723-652b-434f-bad2-ce1c0e3f372a, 15, Finished, Available)

### Get list of available schools

In [17]:
dfSchoolIds = (spark.read
                .format("delta")
                .load(f"{BronzeBasePath}/Schools"))
display(dfSchoolIds)

StatementMeta(, e8215b12-1c89-4e48-9a67-2ae51d02e83a, 40, Finished, Available)

SynapseWidget(Synapse.DataFrame, 38cdb409-703a-4d53-890a-3946e36dda6f)

### Iterate through schools and get groups

In [18]:
def get_groups():
    for school in dfSchoolIds.collect():
        schoolId = school["id"]
        url = f"{baseWondeUrl}/schools/{schoolId}/groups?include=students"
        df = getWondeAPI(url, wondeHeaders, group_schema)
        df = df.withColumn("school_id", lit(school["id"]))
    
        yield df

StatementMeta(, e8215b12-1c89-4e48-9a67-2ae51d02e83a, 41, Finished, Available)

### Write groups and group students to delta lake

In [24]:
# This pulls down all available data and overwrites. Historic versions available still due to Delta Timetravel.

#If full reset needed run:
#try:
#    mssparkutils.fs.rm(f"{BronzeBasePath}/Students", True)
#except:
#    print("Path doesn't exist, most likely already deleted.")

# Write new school data into delta
for school in get_groups():
    dfGroupFinal = (school.select(
                col("school_id"),
                col("id").alias("group_id"),
                col("name"),
                col("code"),
                col("type"),
                col("description"),
                col("notes"),
                col("division"),
                to_timestamp(col("created_at.date"), "yyyy-MM-dd HH:mm:ss.SSSSSS").alias("created_at_date"),
                to_timestamp(col("updated_at.date"), "yyyy-MM-dd HH:mm:ss.SSSSSS").alias("updated_at_date"),
                to_timestamp(col("restored_at.date"), "yyyy-MM-dd HH:mm:ss.SSSSSS").alias("restored_at_date"),
                ))
    dfGroupStudentsFinal = (school.select(
                col("school_id"),
                col("id").alias("group_id"),
                explode(col("students.data.id").alias("student_id"))
                ))

    currentSchoolId = school.first()["school_id"]

    (dfGroupFinal.write
        .mode("overwrite")
        .format("delta")
        .partitionBy("school_id")
        .option("replaceWhere", f"school_id = '{currentSchoolId}'")
        .save(f"{BronzeBasePath}/Groups")
    )

    (dfGroupStudentsFinal.write
        .mode("overwrite")
        .format("delta")
        .partitionBy("school_id")
        .option("replaceWhere", f"school_id = '{currentSchoolId}'")
        .save(f"{BronzeBasePath}/GroupStudents")
    )

    print(f"Completed {currentSchoolId}")


StatementMeta(, e8215b12-1c89-4e48-9a67-2ae51d02e83a, 47, Finished, Available)

Completed A1930499544
Completed A1128526306
